In [1]:
!pip list

Package               Version
--------------------- -----------
argon2-cffi           21.3.0
argon2-cffi-bindings  21.2.0
asttokens             2.0.8
attrs                 22.1.0
backcall              0.2.0
beautifulsoup4        4.11.1
bleach                5.0.1
certifi               2022.6.15
cffi                  1.15.1
charset-normalizer    2.1.1
colorama              0.4.5
cycler                0.11.0
debugpy               1.6.3
decorator             5.1.1
defusedxml            0.7.1
entrypoints           0.4
executing             0.10.0
fastjsonschema        2.16.1
fonttools             4.37.0
htmlmin               0.1.12
idna                  3.3
ImageHash             4.2.1
importlib-metadata    4.12.0
ipykernel             6.15.1
ipython               8.4.0
ipython-genutils      0.2.0
ipywidgets            8.0.1
jedi                  0.18.1
Jinja2                3.1.2
joblib                1.1.0
jsonschema            4.14.0
jupyter-client        7.3.4
jupyter-core          4.11


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import json
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [3]:
INPUT_PATH="input"
OUTPUT_PATH="output"
CONFIG_PATH="config"

INPUT_DATA="test_data2"
INPUT_DATA_CONFIG="input_config"
OUTPUT_DATA="output"

INPUT_EXTENSION="csv"
INPUT_CONFIG_EXTENSION="json"
OUTPUT_EXTENSION="csv"

INPUT_FILE=f"{INPUT_DATA}.{INPUT_EXTENSION}"
INPUT_CONFIG_FILE=f"{INPUT_DATA_CONFIG}.{INPUT_CONFIG_EXTENSION}"
OUTPUT_FILE=f"{OUTPUT_DATA}.{OUTPUT_EXTENSION}"

INPUT_ABS_APTH=os.path.abspath(os.path.join(INPUT_PATH, INPUT_FILE))
INPUT_FILE_CONFIG=os.path.abspath(os.path.join(CONFIG_PATH, INPUT_CONFIG_FILE))
OUTPUT_ABS_APTH=os.path.abspath(os.path.join(OUTPUT_PATH, OUTPUT_FILE))

In [4]:
with open(INPUT_FILE_CONFIG, encoding='utf-8') as f:
    CONFIG = json.load(f)

In [5]:
CONFIG

{'INPUTS': {'FILE_NAME': ['test_data2.csv'],
  'SEPARATOR': ',',
  'DECIMAL': None,
  'ENCODING': 'utf-8',
  'FLOAT_PRECISION': 'high',
  'INDEXES': ['period_end_date', 'translated_when'],
  'DATE_COLUMNS': ['period_end_date', 'translated_when'],
  'DTYPE': {'if_data_corrected': 'object',
   'prod_gr_id': 'object',
   'country_id_n': 'object',
   'delivery_type_id': 'object',
   'freq_id': 'object',
   'retailer_id': 'object',
   'brand_id': 'object',
   'predict_automatch': 'object',
   'class_acctual': 'object'},
  'COLUMNS_WITH_NAN_VALUES': [None]},
 'MODEL': [None],
 'OUTPUTS': {'NAME': ['output.csv']}}

# Functions

In [95]:
def get_DPPL(data_frame, column_to_group_by, column_to_count):
    """
    :param data_frame: 
    :param column_to_group_by: 
    :param column_to_count: 
    :return: 
    """
    
    counts_total = data_frame.groupby([column_to_group_by]) \
    .size() \
    .rename('count_total') \
    .reset_index() \
    .sort_values(by=column_to_group_by, ascending=False) \
    .set_index(column_to_group_by)
    
    
    counts_grouped = data_frame.groupby([column_to_group_by, column_to_count]) \
    .size() \
    .rename('count') \
    .reset_index() \
    .sort_values(by=column_to_group_by, ascending=False) \
    .set_index(column_to_group_by)

    positive_counts = counts_grouped.loc[counts_grouped[column_to_count] == '1']
    df = pd.merge(positive_counts, counts_total, left_index=True, right_index=True)
    df['DPPL'] = df['count'] / df['count_total']

    for cat, dppl in df['DPPL'].iteritems():
        print('Positive Proportion of Predicted Label (PPL) for Variable [prod_gr_id] for Class ==', cat, ':', f'{dppl:.0%}')
            
    return df['DPPL']

# Load data

In [96]:
# LOAD CSV
df = pd.read_csv(
    INPUT_ABS_APTH,
#     index_col=CONFIG['INPUTS']['INDEXES'],
    sep=CONFIG['INPUTS']['SEPARATOR'],
    # quotechar='"',
    # thousands=',',
#     decimal=CONFIG['INPUTS']['DECIMAL'],
    encoding=CONFIG['INPUTS']['ENCODING'],
    infer_datetime_format=True,
    parse_dates=CONFIG['INPUTS']['DATE_COLUMNS'],
    engine="c",
    low_memory=False,
#     float_precision=CONFIG['INPUTS']['DATE_COLUMNS'],
    skipinitialspace=True,
    dtype=CONFIG['INPUTS']['DTYPE']
)

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19697 entries, 0 to 19696
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   period_end_date    19640 non-null  datetime64[ns, UTC]
 1   translated_when    19697 non-null  datetime64[ns, UTC]
 2   if_data_corrected  19697 non-null  object             
 3   prod_gr_id         19697 non-null  object             
 4   country_id_n       18405 non-null  object             
 5   delivery_type_id   18362 non-null  object             
 6   freq_id            19697 non-null  object             
 7   retailer_id        19697 non-null  object             
 8   brand_id           19697 non-null  object             
 9   predict_automatch  19368 non-null  object             
 10  class_acctual      19697 non-null  object             
dtypes: datetime64[ns, UTC](2), object(9)
memory usage: 1.7+ MB


In [98]:
df.head(3)

,period_end_date,translated_when,if_data_corrected,prod_gr_id,country_id_n,delivery_type_id,freq_id,retailer_id,brand_id,predict_automatch,class_acctual
0,NaT,2020-10-15 06:58:28+00:00,0,426,121,NaN,2,96,111,1,1
1,NaT,2020-10-15 06:38:34+00:00,0,426,NaN,NaN,2,92,95,0,1
2,2020-09-20 00:00:00+00:00,2020-09-23 12:27:51+00:00,0,426,121,42730,2,30,37,1,1


In [99]:
df.describe(
    include='all',
    datetime_is_numeric=True
)

,period_end_date,translated_when,if_data_corrected,prod_gr_id,country_id_n,delivery_type_id,freq_id,retailer_id,brand_id,predict_automatch,class_acctual
count,19640,19697,19697,19697,18405,18362,19697,19697,19697,19368,19697
unique,NaN,NaN,2,3,34,914,2,52,199,2,2
top,NaN,NaN,0,426,121,31480,2,30,33,1,1
freq,NaN,NaN,17085,11844,4153,998,11934,1197,587,12324,14639
mean,2020-10-14 23:30:22.729124096+00:00,2020-10-22 06:01:21.525206784+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2020-08-30 00:00:00+00:00,2020-09-01 03:05:51+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2020-09-27 00:00:00+00:00,2020-09-29 13:45:29+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2020-10-18 00:00:00+00:00,2020-10-22 04:30:49+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2020-11-08 00:00:00+00:00,2020-11-13 11:57:53+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,2020-12-01 00:00:00+00:00,2021-02-01 14:50:49+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Pandas data profiler

In [100]:
profile = ProfileReport(
    df,
    title="Pandas Profiling Report",
    pool_size=0,
    infer_dtypes=False
)

In [101]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

# Post-Processing Bias Metrics for the Trained Model

## Difference in positive proportion in predicted labels (DPPL)

![image info](./docs/images/DPPL.png)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)


### CHARACTERISTICS OF METRIC:
> DPPPL > -1 & DPPPL < 1

> For example, if the model grants loans to 50% of class 2 and to 60% of class 1, then it may be biased against class 2.
We would have to decide whether a 10% difference is material.

## Variable: prod_gr_id

### prod_gr_id total count

In [102]:
total_counts = df.groupby(['prod_gr_id']) \
    .size() \
    .rename('count_total') \
    .reset_index() \
    .sort_values(by='prod_gr_id', ascending=False) \
    .set_index('prod_gr_id')

# total_counts

### prod_gr_id by predicted classes

In [103]:
grouped_counts = df.groupby(['prod_gr_id', 'predict_automatch']) \
    .size() \
    .rename('count') \
    .reset_index() \
    .sort_values(by='prod_gr_id', ascending=False) \
    .set_index('prod_gr_id')

# grouped_counts

In [104]:
positive_grouped_counts = grouped_counts.loc[grouped_counts['predict_automatch'] == '1']

# positive_grouped_counts

In [105]:
pd.merge(positive_grouped_counts, total_counts, left_index=True, right_index=True)

,predict_automatch,count,count_total
prod_gr_id,,,
427,1,2235,3367
426,1,7298,11844
413,1,2791,4486


### DPPL metric

In [106]:
dppl_prod_gr_id = get_DPPL(
    data_frame=df,
    column_to_group_by='prod_gr_id',
    column_to_count='predict_automatch'
)

dppl_prod_gr_id

Positive Proportion of Predicted Label (PPL) for Variable [prod_gr_id] for Class == 427 : 66%
Positive Proportion of Predicted Label (PPL) for Variable [prod_gr_id] for Class == 426 : 62%
Positive Proportion of Predicted Label (PPL) for Variable [prod_gr_id] for Class == 413 : 62%


prod_gr_id
427    0.663796
426    0.616177
413    0.622158
Name: DPPL, dtype: float64

### CONCLUSIONS:

> TODO:

## Variable: country_id_n

### country_id_n total count

In [107]:
total_counts = df.groupby(['country_id_n']) \
    .size() \
    .rename('count_total') \
    .reset_index() \
    .sort_values(by='country_id_n', ascending=False) \
    .set_index('country_id_n')

# total_counts

### country_id_n by predicted classes

In [108]:
grouped_counts = df.groupby(['country_id_n', 'predict_automatch']) \
    .size() \
    .rename('count') \
    .reset_index() \
    .sort_values(by='country_id_n', ascending=False) \
    .set_index('country_id_n')

# grouped_counts

In [109]:
positive_grouped_counts = grouped_counts.loc[grouped_counts['predict_automatch'] == '1']
# positive_grouped_counts

In [110]:
pd.merge(positive_grouped_counts, total_counts, left_index=True, right_index=True)

,predict_automatch,count,count_total
country_id_n,,,
177,1,75,110
176,1,679,1022
160,1,32,47
139,1,249,438
138,1,66,104
136,1,20,29
126,1,922,1354
121,1,2699,4153
116,1,326,502


### DPPL metric

In [111]:
dppl_country_id_n = get_DPPL(
    data_frame=df,
    column_to_group_by='country_id_n',
    column_to_count='predict_automatch'
)

dppl_country_id_n

Positive Proportion of Predicted Label (PPL) for Variable [prod_gr_id] for Class == 177 : 68%
Positive Proportion of Predicted Label (PPL) for Variable [prod_gr_id] for Class == 176 : 66%
Positive Proportion of Predicted Label (PPL) for Variable [prod_gr_id] for Class == 160 : 68%
Positive Proportion of Predicted Label (PPL) for Variable [prod_gr_id] for Class == 139 : 57%
Positive Proportion of Predicted Label (PPL) for Variable [prod_gr_id] for Class == 138 : 63%
Positive Proportion of Predicted Label (PPL) for Variable [prod_gr_id] for Class == 136 : 69%
Positive Proportion of Predicted Label (PPL) for Variable [prod_gr_id] for Class == 126 : 68%
Positive Proportion of Predicted Label (PPL) for Variable [prod_gr_id] for Class == 121 : 65%
Positive Proportion of Predicted Label (PPL) for Variable [prod_gr_id] for Class == 116 : 65%
Positive Proportion of Predicted Label (PPL) for Variable [prod_gr_id] for Class == 114 : 69%
Positive Proportion of Predicted Label (PPL) for Variable [p

country_id_n
177     0.681818
176     0.664384
160     0.680851
139     0.568493
138     0.634615
136     0.689655
126     0.680945
121     0.649892
116     0.649402
114     0.688312
113     0.665432
110     0.649778
109     0.386667
108     0.690469
107     0.672269
106     0.430536
105     0.670455
104     0.699856
103     0.711691
1011    0.731844
1010    0.680288
1002    0.733333
Name: DPPL, dtype: float64

### CONCLUSIONS:

> TODO:

## Disparate (Adverse) Impact (DI)

![image info](./docs/images/DI.png)

[SOURCE](https://pages.awscloud.com/rs/112-TZM-766/images/Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf)

### CHARACTERISTICS OF METRIC:
> **DI >= 0**

> This measure may be considered fair if it resides in the **<0.8; 1.2>** range.

## Variable: prod_gr_id

In [112]:
sorted([abs(i / j) for i in dppl_prod_gr_id for j in dppl_prod_gr_id if i != j])

[0.928263019552467,
 0.9372731071860653,
 0.9903869133078523,
 1.0097063951098064,
 1.0669248827615005,
 1.0772808772260676]

### CONCLUSIONS:

> For each possible relation between Classes of Variable **prod_gr_id** the metric resides between **<0.8; 1.2>**

> We can loosely assume that there is no disparity impact between Classes of Variable **prod_gr_id**

## Variable: country_id_n

In [126]:
di_country_id_n = sorted([abs(i / j) for i in dppl_country_id_n for j in dppl_country_id_n if i != j])

In [129]:
print(pd.cut(di_country_id_n, bins=4).value_counts())

(0.526, 0.87]      57
(0.87, 1.212]     359
(1.212, 1.554]     14
(1.554, 1.897]     32
dtype: int64


In [131]:
pd.Series(di_country_id_n).describe()

count    462.000000
mean       1.025311
std        0.238095
min        0.527273
25%        0.943997
50%        1.000000
75%        1.059325
max        1.896552
dtype: float64

### CONCLUSIONS:

> There is more than 100 Classes of Variable **country_id_n** out of **<0.8; 1.2>** bounds.

> At this moment we can loosely assume that there are some examples of Disparity Impact in case of **country_id_n**